<a href="https://colab.research.google.com/github/thanh727/rational-primer-design/blob/main/Application_primer_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🛠️ Install & Setup
# @markdown Run this cell to install the application and dependencies

import os
import sys

# 1. Install System Dependencies
print("⏳ Installing NCBI BLAST+...")
# We use '> /dev/null' to hide success messages, but keep errors visible
!apt-get update -qq > /dev/null
!apt-get install -y ncbi-blast+ > /dev/null

# 2. Clone Repository
if not os.path.exists("rational-primer-design"):
    print("⏳ Cloning Repository...")
    # The -q flag keeps git quiet
    !git clone -q https://github.com/thanh727/rational-primer-design.git

# 3. Enter the Directory (Persistent)
try:
    %cd rational-primer-design
except:
    print("❌ Error: Could not find the repository folder.")

# 4. Install Python Libraries
print("⏳ Installing Python Dependencies...")
# The -q flag keeps pip quiet
!pip install -q -r requirements.txt
!pip install -q biopython pandas primer3-py tqdm

# 5. Create Database Folders (Just in case)
os.makedirs("database/target", exist_ok=True)
os.makedirs("database/background", exist_ok=True)
os.makedirs("config_files", exist_ok=True)

print("\n✅ Setup Complete! Now choose Option A or Option B below.")

In [ ]:
# @title  📂 Option A: Fully automatic design
# @markdown Run this app to download data from NCBI and design primers.

import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import os
import sys
import subprocess
import shutil
from google.colab import files, drive

# --- GUI ELEMENTS ---

header = widgets.HTML("<h2>🚀 Automatic Primer Design App</h2>")

# Inputs
email_input = widgets.Text(description="NCBI Email:", placeholder="your_email@example.com", style={'description_width': 'initial'})
project_input = widgets.Text(description="Project Name:", value="Auto_Design_Run", style={'description_width': 'initial'})
output_input = widgets.Text(description="Output Folder:", value="results_auto", style={'description_width': 'initial'})
blast_checkbox = widgets.Checkbox(value=False, description='Enable Target Gene Annotation (BLAST)', indent=False)

# --- ADVANCED SETTINGS ---
style = {'description_width': 'initial'}
layout_half = widgets.Layout(width='48%')

# Biological Params
w_min_sens = widgets.FloatSlider(value=90.0, min=50.0, max=100.0, step=0.1, description='Min Sensitivity (%)', style=style)
w_min_cons = widgets.FloatSlider(value=0.90, min=0.50, max=1.00, step=0.01, description='Min Conservation (0-1)', style=style)
w_max_xr = widgets.FloatSlider(value=5.0, min=0.0, max=100.0, step=0.1, description='Max Cross-Reactivity (%)', style=style)
w_prod_min = widgets.IntText(value=120, description='Min Product (bp)', style=style, layout=layout_half)
w_prod_max = widgets.IntText(value=400, description='Max Product (bp)', style=style, layout=layout_half)
w_primer_len = widgets.IntText(value=20, description='Primer Length (bp)', style=style, layout=layout_half)
w_max_mm = widgets.IntText(value=2, description='Max Mismatches', style=style, layout=layout_half)
w_cpu = widgets.IntText(value=2, description='CPU Cores (0=Auto)', style=style, layout=layout_half)

# Sampling Params
w_samp_des_t = widgets.IntText(value=0, description='Design Target (0=All)', style=style, layout=layout_half)
w_samp_des_b = widgets.IntText(value=50, description='Design Bg (Speed)', style=style, layout=layout_half)
w_samp_val_t = widgets.IntText(value=0, description='Validate Target (0=All)', style=style, layout=layout_half)
w_samp_val_b = widgets.IntText(value=0, description='Validate Bg (Accuracy)', style=style, layout=layout_half)

advanced_ui = widgets.Accordion(children=[widgets.VBox([
    widgets.HTML("<b>🧬 Biological Parameters:</b>"), w_min_sens, w_min_cons, w_max_xr,
    widgets.HBox([w_prod_min, w_prod_max]), widgets.HBox([w_primer_len, w_max_mm]),
    widgets.HTML("<hr><b>💻 System & Sampling:</b>"), widgets.HBox([w_cpu, w_samp_des_t]),
    widgets.HBox([w_samp_des_b, w_samp_val_t]), widgets.HBox([w_samp_val_b])
])])
advanced_ui.set_title(0, '⚙️ Advanced Configuration'); advanced_ui.selected_index = None

# Search Boxes
target_container = widgets.VBox([]); background_container = widgets.VBox([])
def create_search_box(placeholder): return widgets.Text(placeholder=placeholder, layout=widgets.Layout(width='80%'))
add_target_btn = widgets.Button(description="+ Add Target Query", icon="plus", button_style='success')
add_bg_btn = widgets.Button(description="+ Add Background Query", icon="plus", button_style='warning')
def on_add_target(b): target_container.children += (create_search_box("e.g. Salmonella enterica[Org] AND complete genome"),)
def on_add_bg(b): background_container.children += (create_search_box("e.g. Escherichia coli[Org] AND complete genome"),)
add_target_btn.on_click(on_add_target); add_bg_btn.on_click(on_add_bg); on_add_target(None); on_add_bg(None)

# --- SAVING CONTROLS ---
save_drive_input = widgets.Text(description="Drive Folder:", value="Rational_Design_Results/My_Run", placeholder="Folder Name in Drive", style=style)
save_drive_btn = widgets.Button(description="💾 Save to Drive (Fast)", button_style='success', disabled=True, layout=layout_half)
download_zip_btn = widgets.Button(description="⬇️ Download ZIP (Slow)", button_style='info', disabled=True, layout=layout_half)

run_btn = widgets.Button(description="▶ RUN PIPELINE", button_style='danger', layout=widgets.Layout(width='100%', margin='20px 0px'))
log_output = widgets.Output(layout={'border': '1px solid #ddd', 'height': '400px', 'overflow_y': 'scroll', 'font_family': 'monospace'})

# --- LOGIC ---
def run_command_live(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True, text=True, bufsize=1)
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None: break
        if line: print(line.strip())
    return process.poll()

def on_run_click(b):
    log_output.clear_output(); run_btn.disabled = True; save_drive_btn.disabled = True; download_zip_btn.disabled = True
    run_btn.description = "⏳ Running..."

    with log_output:
        print("⏳ Initializing..."); email = email_input.value; project = project_input.value; folder = output_input.value
        targets = [w.value for w in target_container.children if w.value.strip()]
        backgrounds = [w.value for w in background_container.children if w.value.strip()]

        if not email or "@" not in email: print("❌ Error: Invalid email."); run_btn.disabled = False; return
        if not targets or not backgrounds: print("❌ Error: Missing queries."); run_btn.disabled = False; return

        # Configs
        t_conf = {f"t_{i}": [q, 0.0] for i, q in enumerate(targets)}; b_conf = {f"b_{i}": [q, 0.0] for i, q in enumerate(backgrounds)}
        os.makedirs("config_files", exist_ok=True)
        user_params = {
            "enable_blast": blast_checkbox.value, "min_sensitivity": w_min_sens.value,
            "design_min_conservation": w_min_cons.value, "validation_max_cross_reactivity": w_max_xr.value,
            "product_size_min": w_prod_min.value, "product_size_max": w_prod_max.value,
            "primer_length": w_primer_len.value, "max_mismatch": w_max_mm.value, "cpu_cores": w_cpu.value,
            "design_target_sampling_size": w_samp_des_t.value, "design_background_sampling_size": w_samp_des_b.value,
            "validation_target_sampling_size": w_samp_val_t.value, "validation_background_sampling_size": w_samp_val_b.value,
            "design_max_candidates": 50
        }
        with open("config_files/target_config.json", "w") as f: json.dump(t_conf, f, indent=4)
        with open("config_files/bg_config.json", "w") as f: json.dump(b_conf, f, indent=4)
        with open("config_files/user_params.json", "w") as f: json.dump(user_params, f, indent=4)

        cmd = (f"{sys.executable} -u -m rational_design.cli pipeline --out '{folder}' "
               f"--target_config 'config_files/target_config.json' --bg_config 'config_files/bg_config.json' "
               f"--params 'config_files/user_params.json' --email '{email}'")

        print(f"🚀 Launching Pipeline for '{project}'..."); print("-" * 40)
        exit_code = run_command_live(cmd); print("-" * 40)

        if exit_code == 0:
            print(f"✅✅ PIPELINE FINISHED!"); save_drive_btn.disabled = False; download_zip_btn.disabled = False
        else: print(f"❌ PIPELINE FAILED.")
        run_btn.disabled = False; run_btn.description = "▶ RUN PIPELINE"

def on_save_drive_click(b):
    src = output_input.value
    drive_path = save_drive_input.value.strip()
    full_dest = os.path.join("/content/drive/MyDrive", drive_path)

    with log_output:
        print(f"\n💾 Connecting to Google Drive...")
        if not os.path.exists('/content/drive'):
            try: drive.mount('/content/drive')
            except: print("❌ Error: Could not mount Drive."); return

        print(f"   📂 Copying results to: MyDrive/{drive_path} ...")
        if os.path.exists(full_dest): shutil.rmtree(full_dest) # Overwrite
        shutil.copytree(src, full_dest)
        print(f"   ✅ Success! Files saved to your Google Drive.")

def on_download_zip_click(b):
    folder = output_input.value; zip_name = f"{folder}.zip"
    with log_output:
        print(f"\n📦 Zipping..."); shutil.make_archive(folder, 'zip', folder)
        print(f"⬇️ Downloading..."); files.download(zip_name)

run_btn.on_click(on_run_click); save_drive_btn.on_click(on_save_drive_click); download_zip_btn.on_click(on_download_zip_click)

ui = widgets.VBox([
    header, widgets.HBox([email_input, project_input]), output_input,
    blast_checkbox, advanced_ui,
    widgets.HTML("<hr><h3>🎯 Targets</h3>"), target_container, add_target_btn,
    widgets.HTML("<hr><h3>🌫️ Background</h3>"), background_container, add_bg_btn,
    run_btn,
    widgets.HTML("<hr><h3>💾 Save Results</h3>"),
    widgets.HBox([save_drive_input]),
    widgets.HBox([save_drive_btn, download_zip_btn]),
    log_output
])
display(ui)

In [ ]:
# @title 📂 Option B: Local & Drive Mode (Optimized + Save to Drive)
# @markdown Run this app to use Local Files or Google Drive.
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import sys
import shutil
import json
import subprocess
from google.colab import files, drive

# --- GUI SETUP ---
header = widgets.HTML("<h2>📂 Local & Drive Pipeline Manager</h2>")

# Advanced Settings
style = {'description_width': 'initial'}; layout_half = widgets.Layout(width='48%')
w_min_sens = widgets.FloatSlider(value=90.0, min=50.0, max=100.0, step=0.1, description='Min Sensitivity (%)', style=style)
w_min_cons = widgets.FloatSlider(value=0.90, min=0.50, max=1.00, step=0.01, description='Min Conservation (0-1)', style=style)
w_max_xr = widgets.FloatSlider(value=5.0, min=0.0, max=100.0, step=0.1, description='Max Cross-Reactivity (%)', style=style)
w_prod_min = widgets.IntText(value=120, description='Min Product (bp)', style=style, layout=layout_half)
w_prod_max = widgets.IntText(value=400, description='Max Product (bp)', style=style, layout=layout_half)
w_primer_len = widgets.IntText(value=20, description='Primer Length (bp)', style=style, layout=layout_half)
w_max_mm = widgets.IntText(value=2, description='Max Mismatches', style=style, layout=layout_half)
w_cpu = widgets.IntText(value=2, description='CPU Cores (0=Auto)', style=style, layout=layout_half)
w_samp_des_t = widgets.IntText(value=0, description='Design Target (0=All)', style=style, layout=layout_half)
w_samp_des_b = widgets.IntText(value=50, description='Design Bg (Speed)', style=style, layout=layout_half)
w_samp_val_t = widgets.IntText(value=0, description='Validate Target (0=All)', style=style, layout=layout_half)
w_samp_val_b = widgets.IntText(value=0, description='Validate Bg (Accuracy)', style=style, layout=layout_half)

advanced_ui = widgets.Accordion(children=[widgets.VBox([
    widgets.HTML("<b>🧬 Biological Parameters:</b>"), w_min_sens, w_min_cons, w_max_xr,
    widgets.HBox([w_prod_min, w_prod_max]), widgets.HBox([w_primer_len, w_max_mm]),
    widgets.HTML("<hr><b>💻 System & Sampling:</b>"), widgets.HBox([w_cpu, w_samp_des_t]),
    widgets.HBox([w_samp_des_b, w_samp_val_t]), widgets.HBox([w_samp_val_b])
])])
advanced_ui.set_title(0, '⚙️ Advanced Configuration'); advanced_ui.selected_index = None

# Tabs
tab_nest = widgets.Tab()
desc_local = widgets.HTML("<b>Instructions:</b> Select your FASTA files below.")
upload_target = widgets.FileUpload(accept='.fasta,.fa', multiple=True, description='Select Target Files', button_style='success')
upload_bg = widgets.FileUpload(accept='.fasta,.fa', multiple=True, description='Select Background Files', button_style='warning')
lbl_target = widgets.Label("No files selected"); lbl_bg = widgets.Label("No files selected")
def on_upload_change(change):
    uploader = change['owner']; count = len(uploader.value)
    if uploader == upload_target: lbl_target.value = f"✅ Ready to upload {count} Target files"
    else: lbl_bg.value = f"✅ Ready to upload {count} Background files"
upload_target.observe(on_upload_change, names='value'); upload_bg.observe(on_upload_change, names='value')
tab1 = widgets.VBox([desc_local, widgets.HBox([upload_target, lbl_target]), widgets.HBox([upload_bg, lbl_bg])])

desc_drive = widgets.HTML("<b>Instructions:</b> Mount Drive, then enter your folder paths.")
btn_mount = widgets.Button(description="🔌 Mount Google Drive", icon="google", button_style='info')
path_target_input = widgets.Text(description="Target Path:", placeholder="/content/drive/MyDrive/...", layout=widgets.Layout(width='80%'))
path_bg_input = widgets.Text(description="Bg Path:", placeholder="/content/drive/MyDrive/...", layout=widgets.Layout(width='80%'))
def on_mount_click(b):
    try: drive.mount('/content/drive'); btn_mount.description = "✅ Drive Mounted"; btn_mount.disabled = True
    except Exception as e: btn_mount.description = "❌ Mount Failed"; print(e)
btn_mount.on_click(on_mount_click)
tab2 = widgets.VBox([desc_drive, btn_mount, path_target_input, path_bg_input])
tab_nest.children = [tab1, tab2]; tab_nest.set_title(0, '💻 Upload from Computer'); tab_nest.set_title(1, '☁️ Use Google Drive')

# Controls
project_input = widgets.Text(description="Project Name:", value="My_Local_Run")
output_input = widgets.Text(description="Output Folder:", value="results_local")
blast_checkbox = widgets.Checkbox(value=False, description='Enable Target Gene Annotation (BLAST)', indent=False)

# Saving Controls
save_drive_input = widgets.Text(description="Drive Folder:", value="Rational_Design_Results/My_Local_Run", placeholder="Folder Name in Drive", style=style)
save_drive_btn = widgets.Button(description="💾 Save to Drive (Fast)", button_style='success', disabled=True, layout=layout_half)
download_zip_btn = widgets.Button(description="⬇️ Download ZIP (Slow)", button_style='info', disabled=True, layout=layout_half)

run_btn = widgets.Button(description="▶ RUN PIPELINE", button_style='danger', layout=widgets.Layout(width='100%', margin='20px 0px'))
log_output = widgets.Output(layout={'border': '1px solid #ddd', 'height': '400px', 'overflow_y': 'scroll', 'font_family': 'monospace'})

# --- LOGIC ---
def copy_folder_contents(src_folder, dest_folder):
    os.makedirs(dest_folder, exist_ok=True); files_list = [f for f in os.listdir(src_folder) if f.endswith(('.fasta', '.fa'))]
    count = 0; print(f"   📂 Found {len(files_list)} files in {src_folder}...")
    for f in files_list: shutil.copy2(os.path.join(src_folder, f), os.path.join(dest_folder, f)); count += 1
    return count

def save_uploaded_files(uploader, dest_folder):
    os.makedirs(dest_folder, exist_ok=True); count = 0
    for file_info in uploader.value:
        try:
            with open(os.path.join(dest_folder, file_info.get('name')), 'wb') as f: f.write(file_info.get('content'))
            count += 1
        except Exception as e: print(f"❌ Error saving {file_info.get('name')}: {e}")
    return count

def run_command_live(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True, text=True, bufsize=1)
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None: break
        if line: print(line.strip())
    return process.poll()

def on_run_click(b):
    log_output.clear_output(); run_btn.disabled = True; save_drive_btn.disabled = True; download_zip_btn.disabled = True
    run_btn.description = "⏳ Processing..."; selected_tab = tab_nest.selected_index; folder_out = output_input.value
    fast_target_dir = "temp_fast_target"; fast_bg_dir = "temp_fast_bg"
    if os.path.exists(fast_target_dir): shutil.rmtree(fast_target_dir)
    if os.path.exists(fast_bg_dir): shutil.rmtree(fast_bg_dir)

    with log_output:
        print("⏳ Initializing Workspace...")
        os.makedirs("config_files", exist_ok=True)
        user_params = {
            "enable_blast": blast_checkbox.value, "min_sensitivity": w_min_sens.value,
            "design_min_conservation": w_min_cons.value, "validation_max_cross_reactivity": w_max_xr.value,
            "product_size_min": w_prod_min.value, "product_size_max": w_prod_max.value,
            "primer_length": w_primer_len.value, "max_mismatch": w_max_mm.value, "cpu_cores": w_cpu.value,
            "design_target_sampling_size": w_samp_des_t.value, "design_background_sampling_size": w_samp_des_b.value,
            "validation_target_sampling_size": w_samp_val_t.value, "validation_background_sampling_size": w_samp_val_b.value,
            "design_max_candidates": 50
        }
        with open("config_files/local_params.json", "w") as f: json.dump(user_params, f, indent=4)

        try:
            if selected_tab == 0: # UPLOAD
                print("💻 Mode: Local Upload")
                if not upload_target.value or not upload_bg.value: print("❌ Error: Missing files."); raise Exception("Missing files")
                print("   ⬇ Uploading..."); n_t = save_uploaded_files(upload_target, fast_target_dir); n_b = save_uploaded_files(upload_bg, fast_bg_dir)
                print(f"   ✅ Staged {n_t} Target and {n_b} Background files.")
            elif selected_tab == 1: # DRIVE
                print("☁️ Mode: Google Drive (Copying...)")
                t_src = path_target_input.value.strip(); b_src = path_bg_input.value.strip()
                if not os.path.exists(t_src) or not os.path.exists(b_src): print(f"❌ Error: Invalid paths."); raise Exception("Invalid Path")
                n_t = copy_folder_contents(t_src, fast_target_dir); n_b = copy_folder_contents(b_src, fast_bg_dir)
                print(f"   ✅ Copied {n_t} Target and {n_b} Background files.")

            cmd = (f"{sys.executable} -u -m rational_design.cli pipeline --out '{folder_out}' "
                   f"--local_target '{fast_target_dir}' --local_bg '{fast_bg_dir}' --params 'config_files/local_params.json'")

            print(f"🚀 Launching Pipeline..."); print("-" * 40)
            exit_code = run_command_live(cmd); print("-" * 40)
            if exit_code == 0: print(f"✅✅ PIPELINE FINISHED!"); save_drive_btn.disabled = False; download_zip_btn.disabled = False
            else: print(f"❌ PIPELINE FAILED.")

        except Exception as e: print(f"❌ Error: {e}")
        run_btn.disabled = False; run_btn.description = "▶ RUN PIPELINE"

def on_save_drive_click(b):
    src = output_input.value; drive_path = save_drive_input.value.strip()
    full_dest = os.path.join("/content/drive/MyDrive", drive_path)
    with log_output:
        print(f"\n💾 Connecting to Drive...")
        if not os.path.exists('/content/drive'):
            try: drive.mount('/content/drive')
            except: print("❌ Error: Could not mount Drive."); return
        print(f"   📂 Copying to: MyDrive/{drive_path} ...");
        if os.path.exists(full_dest): shutil.rmtree(full_dest)
        shutil.copytree(src, full_dest); print(f"   ✅ Saved to Google Drive.")

def on_download_zip_click(b):
    folder = output_input.value; zip_name = f"{folder}.zip"
    with log_output:
        print(f"\n📦 Zipping..."); shutil.make_archive(folder, 'zip', folder)
        print(f"⬇️ Downloading..."); files.download(zip_name)

run_btn.on_click(on_run_click); save_drive_btn.on_click(on_save_drive_click); download_zip_btn.on_click(on_download_zip_click)

ui = widgets.VBox([
    header, widgets.HBox([project_input, output_input]), blast_checkbox, advanced_ui,
    tab_nest, run_btn,
    widgets.HTML("<hr><h3>💾 Save Results</h3>"),
    widgets.HBox([save_drive_input]),
    widgets.HBox([save_drive_btn, download_zip_btn]),
    log_output
])
display(ui)